In [120]:
from reportlab.lib.pagesizes import A4
from reportlab.platypus import SimpleDocTemplate, Paragraph, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.lib.enums import TA_CENTER, TA_JUSTIFY
from datetime import datetime
from time import time
from os.path import isfile
import PIL

In [121]:
def generateTimeStamp() -> str:
    date_time = datetime.fromtimestamp(time())
    timestamp = date_time.strftime("%d-%m-%Y %H:%M")
    return timestamp


def getNewFileWidth(filename:str, definedheight:int) -> int:
    """This method calculates and returns a new width for an image, given the image file name and the desired heigth, while preserving aspect ratio."""    
    im = PIL.Image.open(filename)
    w, h = im.size
    im.close()

    ratio = w / h
    newwidth = int(definedheight * ratio)
    
    return newwidth


def createTitleParagraph(content:str, styles) -> Paragraph:
    paragraph = Paragraph(text=content, style=styles["Title"])
    return paragraph


def createTextParagraph(content:str, styles) -> Paragraph:
    paragraph = Paragraph(text=content, style=styles["NormalJustified"])
    return paragraph


def createTimestampParagraph(content:str, styles, lang:str) -> Paragraph:

    if lang == "eng":
        content = f"Generated {content}"
    elif lang == "est":
        content = f"Genereeritud {content}"
    else:
        #default to eng
        content = f"Generated {content}"

    paragraph = Paragraph(text=content, style=styles["Heading5"])
    return paragraph


def createImageReferenceParagraph(content:str, styles, lang:str) -> Paragraph:

    content = content.strip()
    content = content[:len(content) - 4]
    contents = content.split("_")

    number = contents[len(contents) - 1]
    
    if lang == "eng":
        content = f"Image {number}"
    elif lang == "est":
        content = f"Pilt {number}"
    else:
        #default to eng
        content = f"Image {number}"

    paragraph = Paragraph(text=content, style=styles["Reference"])

    return paragraph


def createImageParagraph(filename:str, desiredheight:int) -> Image:

    w = getNewFileWidth(filename=filename, definedheight=desiredheight)

    image = Image(filename=filename, 
                  height=desiredheight,
                  width=w,
                  hAlign="CENTER",
                  lazy=2)

    return image


def createPDF(filename:str, lang:str, header:str, text:str, imagefiles:list = None) -> bool:
    """Creates a PDF file with the provided inputs. List of images can be an empty list. Returns true if PDF file was created sucessfully."""
    
    timestamp = generateTimeStamp()
    
    #make sure lang variable is good
    lang = str(lang).lower()
    if lang == "eng" or lang == "en":
        lang = "eng"
    if lang == "est" or lang == "ee" or lang == "et":
        lang = "est"

    #create document
    document = SimpleDocTemplate(
        filename=filename,
        pagesize=A4,
        rightMargin=50, leftMargin=50,
        topMargin=50, bottomMargin=50,
    )

    #get some default styles
    styles = getSampleStyleSheet()

    #define some of my own styles
    styles.add(ParagraphStyle(name='Reference',
                                parent=styles['Normal'],
                                fontSize=12,
                                spaceBefore=2,
                                spaceAfter=12,
                                alignment=TA_CENTER))
    
    styles.add(ParagraphStyle(name='NormalJustified',
                                parent=styles['Normal'],
                                fontSize=12,
                                spaceAfter=50,
                                alignment=TA_JUSTIFY))
    
    flowables = []
    
    #add the title
    flowables.append(createTitleParagraph(content=header, styles=styles))

    #add time created
    flowables.append(createTimestampParagraph(content=timestamp, styles=styles, lang=lang))

    #add summary
    flowables.append(createTextParagraph(content=text, styles=styles))

    if not imagefiles is None or len(imagefiles) > 1:
        #add all image files with their references, if they exist
        for imagefilename in imagefiles:
            if isfile(imagefilename):
                flowables.append(createImageParagraph(filename=imagefilename, desiredheight=150))
                flowables.append(createImageReferenceParagraph(content=imagefilename, styles=styles, lang=lang))

        #build the document
        document.build(flowables=flowables)
    
    return True

In [122]:
#testing

filename = "testfail.pdf"
filename2 = ""

header = "Heading of the document"
header2 = ""

text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum."
text2 = ""

images = ["file_1.png", "file_2.png", "file_3.png"]
images2 = ["Picture_0.png", "Picture_1.png", "Picture_2.png", "Picture_3.png", "Picture_4.png"]
images3 = []

language = "est"

createPDF(filename=filename, lang=language, header=header, text=text, imagefiles=images2)

True